In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Modellare un fluido non viscoso in movimento usando QUICK-PDE

> **Note:** Le Qiskit Functions sono una funzionalità sperimentale disponibile solo per gli utenti del Piano Premium IBM Quantum&reg;, Piano Flex e Piano On-Prem (tramite IBM Quantum Platform API). Sono in stato di rilascio in anteprima e soggette a modifiche.

*Stima di utilizzo: 50 minuti su un processore Heron r2. (NOTA: Questa è solo una stima. Il tempo di esecuzione effettivo potrebbe variare.)*

Notate che il tempo di esecuzione di questa funzione è generalmente superiore a 20 minuti,
quindi potreste voler dividere questo tutorial in due sezioni: la prima, in cui
lo leggete e lanciate i job, e la seconda qualche ora dopo
(concedendo ampio tempo per il completamento dei job), per lavorare con i risultati dei job.
## Contesto
Questo tutorial cerca di insegnare a livello introduttivo come utilizzare la funzione QUICK-PDE
per risolvere problemi multifisici complessi su QPU Heron R2 da 156Q utilizzando
l'H-DES (Hybrid Differential Equation Solver) di ColibriTD.
L'algoritmo sottostante è descritto nel [paper sull'H-DES.](https://arxiv.org/abs/2410.01130)
Notate che questo risolutore può anche risolvere equazioni non lineari.

I problemi multifisici - tra cui dinamica dei fluidi, diffusione del calore e
deformazione dei materiali, solo per citarne alcuni - possono essere descritti ubiquitariamente da Equazioni
Differenziali Parziali (PDE).

Tali problemi sono altamente rilevanti per varie industrie e costituiscono un
importante ramo della matematica applicata. Tuttavia, risolvere PDE accoppiate
multivariabili non lineari con strumenti classici rimane una sfida a causa del
requisito di una quantità esponenzialmente grande di risorse.

Questa funzione è appropriata per equazioni con crescente complessità e
variabili, ed è il primo passo per sbloccare possibilità che un tempo erano
considerate intrattabili. Per descrivere completamente un problema modellato da PDE, è
necessario conoscere le condizioni iniziali e al contorno. Queste possono fortemente
modificare la soluzione della PDE e il percorso per trovare la loro soluzione.

Questo tutorial vi insegna come:

1. Definire i parametri della funzione di condizione iniziale.
2. Regolare il numero di qubit (utilizzati per codificare la funzione dell'equazione differenziale), la profondità e il numero di shot.
3. Eseguire QUICK-PDE per risolvere l'equazione differenziale sottostante.
## Requisiti
Prima di iniziare questo tutorial, assicuratevi di avere installato quanto segue:

* Qiskit SDK v2.0 o successivo (`pip install qiskit`)
* Qiskit Functions Catalog (`pip install qiskit-ibm-catalog`)
* Matplotlib (`pip install matplotlib`)
* Accesso alla funzione QUICK-PDE. Compilate il [modulo per richiedere l'accesso.](https://forms.cloud.microsoft/e/3Wi9cbjQPK)
## Configurazione
Autenticatevi utilizzando la vostra [chiave API](http://quantum.cloud.ibm.com/) e selezionate la funzione come segue:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog(
    channel="ibm_quantum_platform",
    instance="INSTANCE_CRN",
    token="YOUR_API_KEY",  # Use the 44-character API_KEY you created and saved from the IBM Quantum Platform Home dashboard
)

quick = catalog.load("colibritd/quick-pde")

## Step 1: Impostare le proprietà del problema da risolvere
Questo tutorial copre l'esperienza utente da due prospettive: il
problema fisico determinato dalle condizioni iniziali, e la componente
algoritmica nel risolvere un esempio di dinamica dei fluidi su un computer quantistico.

La Computational Fluid Dynamics (CFD) ha una vasta gamma di applicazioni, ed è quindi
importante studiare e risolvere le PDE sottostanti. Una famiglia importante di PDE
sono le equazioni di Navier-Stokes, che sono un sistema di equazioni differenziali
parziali non lineari che descrivono il moto dei fluidi. Sono altamente rilevanti
per problemi scientifici e applicazioni ingegneristiche.

In determinate condizioni, le equazioni di Navier-Stokes si riducono all'equazione
di Burgers, un'equazione di convezione-diffusione che descrive fenomeni che si verificano nella
dinamica dei fluidi, nella dinamica dei gas e nell'acustica non lineare, solo per citarne alcuni, modellando
sistemi dissipativi.

La versione unidimensionale dell'equazione dipende da due variabili:
$t \in \mathbb{R}_{\geq 0}$ che modella la dimensione temporale, $x \in \mathbb{R}$
che rappresenta la dimensione spaziale. La forma generale dell'equazione è chiamata
equazione di Burgers viscosa e si legge:

$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial^2 x},$

dove $u(x,t)$ è il campo di velocità del fluido in una data posizione $x$ e tempo $t$, e $\nu$
è la viscosità del fluido. La viscosità è una proprietà importante di un fluido
che misura la sua resistenza dipendente dalla velocità al movimento o alla deformazione, e quindi
gioca un ruolo cruciale nella determinazione della dinamica di un fluido. Quando la
viscosità del fluido è nulla ($\nu$ = 0), l'equazione diventa un'equazione di conservazione
che può sviluppare discontinuità (onde d'urto), a causa della mancanza di
resistenza interna. In questo caso, l'equazione è chiamata equazione di Burgers
non viscosa ed è un caso speciale di equazione d'onda non lineare.

Rigorosamente parlando, i flussi non viscosi non si verificano in natura, ma quando si modella
il flusso aerodinamico, a causa dell'effetto infinitesimale del trasporto, l'utilizzo di una
descrizione non viscosa del problema può essere utile. Sorprendentemente, più del
70% della teoria aerodinamica si occupa di flussi non viscosi.

Questo tutorial utilizza l'equazione di Burgers non viscosa come esempio di CFD da risolvere
sulle QPU IBM&reg; utilizzando QUICK-PDE, data dall'equazione:

$\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = 0 $

La condizione iniziale per questo problema è impostata su una funzione lineare:
$u(t=0,x) = ax + b,\text{ con }a,b\in\mathbb{R}$
dove $a$ e $b$ sono costanti arbitrarie che influenzano la forma della
soluzione. Potete regolare $a$ e $b$ e vedere come influenzano il
processo di risoluzione e la soluzione.

In [ ]:
job = quick.run(
    use_case="cfd",
    physical_parameters={"a": 1.0, "b": 1.0},
)

In [ ]:
print(job.result())

{'functions': {'u': array([[1.        , 0.96112378, 0.9230742 , 0.88616096, 0.85058445,
        0.81644741, 0.78376878, 0.75249908, 0.72253689, 0.69374562,
        0.66597013, 0.63905258, 0.61284684, 0.58723093, 0.56211691,
        0.53745752, 0.51324915, 0.48953036, 0.46637547, 0.44388257,
        0.4221554 , 0.40127848, 0.38128488, 0.36211604, 0.34357308,
        0.32525895, 0.30651089, 0.28632252, 0.26325504, 0.23533692],
       [1.2375    , 1.19267729, 1.14850734, 1.10544526, 1.06382155,
        1.02385326, 0.98565757, 0.94926734, 0.91464784, 0.88171402,
        0.85034771, 0.82041411, 0.79177677, 0.76431068, 0.73791248,
        0.71250742, 0.68805224, 0.66453346, 0.64196021, 0.62035121,
        0.59971506, 0.5800232 , 0.56117499, 0.54295419, 0.52497612,
        0.50662498, 0.48698059, 0.4647339 , 0.43809065, 0.40466247],
       [1.475     , 1.4242308 , 1.37394048, 1.32472956, 1.27705866,
        1.23125911, 1.18754636, 1.1460356 , 1.10675879, 1.06968242,
        1.03472529, 1.0017

## Step 2 (if needed): Optimize problem for quantum hardware execution

By default, the solver uses physically-informed parameters, which are initial circuit parameters for a given qubit number and depth from which our solver will start.

The shots are also part of the parameters with a default value, since fine-tuning them is important.

Depending on the configuration you're trying to solve, the algorithm's
parameters to achieve satisfactory solutions might need to be adapted; for example, it
can require more or fewer qubits per variable $t$ and $x$, depending on $a$ and
$b$. The following adjusts the number of qubits per function per
variable, the depth per function, and the number of shots.

You can also see how to specify the backend and the execution mode.

In addition, physically-informed parameters might steer the optimization process
in a wrong direction; in that case, you can disable it by setting the
`initialization` strategy to `"RANDOM"`.

In [ ]:
job_2 = quick.run(
    use_case="cfd",
    physical_parameters={"a": 0.5, "b": 0.25},
    nb_qubits={"u": {"t": 2, "x": 1}},
    depth={"u": 3},
    shots=[500, 2500, 5000, 10000],
    initialization="RANDOM",
    backend="ibm_kingston",
    mode="session",
)

In [ ]:
print(job_2.result())

{'functions': {'u': array([[0.25      , 0.24856543, 0.24687708, 0.2449444 , 0.24277686,
        0.24038389, 0.23777496, 0.23495952, 0.23194702, 0.22874691,
        0.22536866, 0.22182171, 0.21811551, 0.21425952, 0.2102632 ,
        0.20613599, 0.20188736, 0.19752675, 0.19306361, 0.18850741,
        0.18386759, 0.1791536 , 0.17437491, 0.16954096, 0.16466122,
        0.15974512, 0.15480213, 0.1498417 , 0.14487328, 0.13990632],
       [0.36875   , 0.36681313, 0.36457201, 0.36203594, 0.35921422,
        0.35611615, 0.35275103, 0.34912817, 0.34525687, 0.34114643,
        0.33680614, 0.33224532, 0.32747327, 0.32249928, 0.31733266,
        0.31198271, 0.30645873, 0.30077002, 0.29492589, 0.28893564,
        0.28280857, 0.27655397, 0.27018116, 0.26369944, 0.2571181 ,
        0.25044645, 0.24369378, 0.23686941, 0.22998264, 0.22304275],
       [0.4875    , 0.48506084, 0.48226695, 0.47912748, 0.47565158,
        0.47184841, 0.46772711, 0.46329683, 0.45856672, 0.45354594,
        0.44824363, 0.4426

## Step 3: Compare the algorithm performances

You can compare the convergence process of our solution (HDES) of job_2 to the performance of a physics-informed neural networks (PINN) algorithm and solver (see the [paper](https://arxiv.org/abs/1711.10561) and the associated [GitHub repository](https://github.com/314arhaam/burger-pinn)).

In the example of job_2's output (quantum-based approach), only  13 parameters (12 circuit parameters plus 1 scaling parameter) are optimized with the classical solver.
The convergence process is as follows:

```
optimizers:
   CMA: {'ftarget': np.float64(0.1), 'verb_disp': 10, 'maxiter': 100}
   CMA: {'ftarget': np.float64(0.005), 'verb_disp': 10, 'maxiter': 20}
   CMA: {'ftarget': np.float64(0.0025), 'verb_disp': 10, 'maxiter': 30}
   CMA: {'ftarget': np.float64(0.0005), 'verb_disp': 10, 'maxiter': 10}

500 shots
================== CMA =================
option:  {'ftarget': np.float64(0.1), 'verb_disp': 10, 'maxiter': 100}
0/100, loss: 0.02456641

1000 shots
================== CMA =================
option:  {'ftarget': np.float64(0.005), 'verb_disp': 10, 'maxiter': 20}
0/20, loss: 0.03641833
1/20, loss: 0.02461719
2/20, loss: 0.0283689
3/20, loss: 0.009898383
4/20, loss: 0.04454522
5/20, loss: 0.007019971
6/20, loss: 0.00811147
7/20, loss: 0.01592619
8/20, loss: 0.00764708
9/20, loss: 0.01401516
10/20, loss: 0.01767467
11/20, loss: 0.01220387

5000 shots
================== CMA =================
option:  {'ftarget': np.float64(0.0025), 'verb_disp': 10, 'maxiter': 30}
0/30, loss: 0.01024792
1/30, loss: 0.004343748
2/30, loss: 0.01450951
3/30, loss: 0.008591284
4/30, loss: 0.00266414
5/30, loss: 0.007923613
6/30, loss: 0.02023853
7/30, loss: 0.01031438
8/30, loss: 0.009513116
9/30, loss: 0.008132266
10/30, loss: 0.005787766
11/30, loss: 0.00390582

10000 shots
================== CMA =================
option:  {'ftarget': np.float64(0.0005), 'verb_disp': 10, 'maxiter': 10}
0/10, loss: 0.002386168
1/10, loss: 0.004024823
2/10, loss: 0.001311999
3/10, loss: 0.003433991
4/10, loss: 0.002339664
5/10, loss: 0.002978438
6/10, loss: 0.005458391
7/10, loss: 0.002026701
8/10, loss: 0.00207467
9/10, loss: 0.001947627
final_loss: 0.00151994463476429

```
That means a loss below 0.0015 can be reached after 28 iterations, and with optimizing only a few classical parameters.

Now we can compare the same to the PINN solution with the default configuration suggested by the paper using a gradient-based optimizer. The equivalent of our circuit with 13 parameters to be optimized is the neural network, which requires at least eight layers of 20 neurons, and thus involves optimizing 3021 parameters.
Then, the target loss is reached at Step 315, loss: 0.0014988397.

![Graph showing PINN data compared with the HDES-Qiskit function.](../docs/images/tutorials/colibritd-pde/pinn-data.avif)

Now, since we want to do a fair comparison, we should use the same optimizer in both cases.
The lowest number of iterations we found for 12 layers of 20 neurons = 4701 parameters:
```
(10_w,20)-aCMA-ES (mu_w=5.9,w_1=27%) in dimension 4701 (seed=351961)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     20 5.398521572351456e-02 1.0e+00 9.98e-03  1e-02  1e-02 0:02.3
    2     40 5.444650724530220e-02 1.0e+00 9.97e-03  1e-02  1e-02 0:05.1
    3     60 4.447407275438309e-02 1.0e+00 9.95e-03  1e-02  1e-02 0:08.2
    4     80 2.068969979882240e-02 1.0e+00 9.94e-03  1e-02  1e-02 0:11.7
    6    120 1.028892211616039e-02 1.0e+00 9.91e-03  1e-02  1e-02 0:20.1
    7    140 5.140972323715687e-03 1.0e+00 9.90e-03  1e-02  1e-02 0:25.4
    9    180 3.811701666563749e-03 1.0e+00 9.87e-03  1e-02  1e-02 0:37.4
   10    200 3.189878538250923e-03 1.0e+00 9.85e-03  1e-02  1e-02 0:44.2
   12    240 2.547040116041899e-03 1.0e+00 9.83e-03  1e-02  1e-02 0:59.7
   14    280 2.166548743844032e-03 1.0e+00 9.80e-03  1e-02  1e-02 1:18.0
   15    300 1.783065614290535e-03 1.0e+00 9.79e-03  1e-02  1e-02 1:28.4
   16    320 2.045844215899706e-03 1.0e+00 9.78e-03  1e-02  1e-02 1:39.8
Stopping early: loss 0.001405 <= target 0.0015
CMA-ES finished. Best loss: 0.001404788694344461
```

You can do the same with your data from job_2, and plot a comparison to the PINN solution.

In [ ]:
# check the loss function and compare between the two approaches
print(job_2.logs())

## Step 4: Use the result

With your solution, you can now choose what to do with it. The following demonstrates how to plot the result.

In [ ]:
solution = job.result()

# Plot the solution of the second simulation job_2
_ = plt.figure()
ax = plt.axes(projection="3d")

# plot the solution using the 3d plotting capabilities of pyplot
t, x = np.meshgrid(solution["samples"]["t"], solution["samples"]["x"])
ax.plot_surface(
    t,
    x,
    solution["functions"]["u"],
    edgecolor="royalblue",
    lw=0.25,
    rstride=26,
    cstride=26,
    alpha=0.3,
)
ax.scatter(t, x, solution, marker=".")
ax.set(xlabel="t", ylabel="x", zlabel="u(t,x)")

plt.show()

<Image src="../docs/images/tutorials/colibritd-pde/extracted-outputs/fe0a7d02-0.avif" alt="Output of the previous code cell" />

## Step 3: Confrontare le prestazioni degli algoritmi
Potete confrontare il processo di convergenza della nostra soluzione (HDES) di job_2 con le prestazioni di un algoritmo e risolutore basato su reti neurali informate dalla fisica (PINN) (consultate il [paper](https://arxiv.org/abs/1711.10561) e il [repository GitHub](https://github.com/314arhaam/burger-pinn) associato).

Nell'esempio dell'output di job_2 (approccio basato su quantum), vengono ottimizzati solo 13 parametri (12 parametri del circuito più 1 parametro di scala) con il risolutore classico.
Il processo di convergenza è il seguente:

```
optimizers:
   CMA: {'ftarget': np.float64(0.1), 'verb_disp': 10, 'maxiter': 100}
   CMA: {'ftarget': np.float64(0.005), 'verb_disp': 10, 'maxiter': 20}
   CMA: {'ftarget': np.float64(0.0025), 'verb_disp': 10, 'maxiter': 30}
   CMA: {'ftarget': np.float64(0.0005), 'verb_disp': 10, 'maxiter': 10}

500 shots
================== CMA =================
option:  {'ftarget': np.float64(0.1), 'verb_disp': 10, 'maxiter': 100}
0/100, loss: 0.02456641

1000 shots
================== CMA =================
option:  {'ftarget': np.float64(0.005), 'verb_disp': 10, 'maxiter': 20}
0/20, loss: 0.03641833
1/20, loss: 0.02461719
2/20, loss: 0.0283689
3/20, loss: 0.009898383
4/20, loss: 0.04454522
5/20, loss: 0.007019971
6/20, loss: 0.00811147
7/20, loss: 0.01592619
8/20, loss: 0.00764708
9/20, loss: 0.01401516
10/20, loss: 0.01767467
11/20, loss: 0.01220387

5000 shots
================== CMA =================
option:  {'ftarget': np.float64(0.0025), 'verb_disp': 10, 'maxiter': 30}
0/30, loss: 0.01024792
1/30, loss: 0.004343748
2/30, loss: 0.01450951
3/30, loss: 0.008591284
4/30, loss: 0.00266414
5/30, loss: 0.007923613
6/30, loss: 0.02023853
7/30, loss: 0.01031438
8/30, loss: 0.009513116
9/30, loss: 0.008132266
10/30, loss: 0.005787766
11/30, loss: 0.00390582

10000 shots
================== CMA =================
option:  {'ftarget': np.float64(0.0005), 'verb_disp': 10, 'maxiter': 10}
0/10, loss: 0.002386168
1/10, loss: 0.004024823
2/10, loss: 0.001311999
3/10, loss: 0.003433991
4/10, loss: 0.002339664
5/10, loss: 0.002978438
6/10, loss: 0.005458391
7/10, loss: 0.002026701
8/10, loss: 0.00207467
9/10, loss: 0.001947627
final_loss: 0.00151994463476429

```
Ciò significa che una loss inferiore a 0.0015 può essere raggiunta dopo 28 iterazioni, e ottimizzando solo pochi parametri classici.

Ora possiamo confrontare lo stesso con la soluzione PINN con la configurazione predefinita suggerita dal paper utilizzando un ottimizzatore basato sul gradiente. L'equivalente del nostro circuito con 13 parametri da ottimizzare è la rete neurale, che richiede almeno otto strati di 20 neuroni, e quindi comporta l'ottimizzazione di 3021 parametri.
Poi, la loss target viene raggiunta allo Step 315, loss: 0.0014988397.

![Graph showing PINN data compared with the HDES-Qiskit function.](../docs/images/tutorials/colibritd-pde/pinn-data.avif)

Ora, poiché vogliamo fare un confronto equo, dovremmo usare lo stesso ottimizzatore in entrambi i casi.
Il numero più basso di iterazioni che abbiamo trovato per 12 strati di 20 neuroni = 4701 parametri:
```
(10_w,20)-aCMA-ES (mu_w=5.9,w_1=27%) in dimension 4701 (seed=351961)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     20 5.398521572351456e-02 1.0e+00 9.98e-03  1e-02  1e-02 0:02.3
    2     40 5.444650724530220e-02 1.0e+00 9.97e-03  1e-02  1e-02 0:05.1
    3     60 4.447407275438309e-02 1.0e+00 9.95e-03  1e-02  1e-02 0:08.2
    4     80 2.068969979882240e-02 1.0e+00 9.94e-03  1e-02  1e-02 0:11.7
    6    120 1.028892211616039e-02 1.0e+00 9.91e-03  1e-02  1e-02 0:20.1
    7    140 5.140972323715687e-03 1.0e+00 9.90e-03  1e-02  1e-02 0:25.4
    9    180 3.811701666563749e-03 1.0e+00 9.87e-03  1e-02  1e-02 0:37.4
   10    200 3.189878538250923e-03 1.0e+00 9.85e-03  1e-02  1e-02 0:44.2
   12    240 2.547040116041899e-03 1.0e+00 9.83e-03  1e-02  1e-02 0:59.7
   14    280 2.166548743844032e-03 1.0e+00 9.80e-03  1e-02  1e-02 1:18.0
   15    300 1.783065614290535e-03 1.0e+00 9.79e-03  1e-02  1e-02 1:28.4
   16    320 2.045844215899706e-03 1.0e+00 9.78e-03  1e-02  1e-02 1:39.8
Stopping early: loss 0.001405 <= target 0.0015
CMA-ES finished. Best loss: 0.001404788694344461
```

Potete fare lo stesso con i vostri dati da job_2 e tracciare un confronto con la soluzione PINN.

In [ ]:
solution_2 = job_2.result()

# Plot the solution of the second simulation job_2
_ = plt.figure()
ax = plt.axes(projection="3d")

# plot the solution using the 3d plotting capabilities of pyplot
t, x = np.meshgrid(solution_2["samples"]["t"], solution_2["samples"]["x"])
ax.plot_surface(
    t,
    x,
    solution_2["functions"]["u"],
    edgecolor="royalblue",
    lw=0.25,
    rstride=26,
    cstride=26,
    alpha=0.3,
)
ax.scatter(t, x, solution_2, marker=".")
ax.set(xlabel="t", ylabel="x", zlabel="u(t,x)")

plt.show()

<Image src="../docs/images/tutorials/colibritd-pde/extracted-outputs/6dab21c9-0.avif" alt="Output of the previous code cell" />

## Step 4: Utilizzare il risultato
Con la vostra soluzione, potete ora scegliere cosa farne. Il seguente esempio dimostra come tracciare il risultato.